In [ ]:
import pandas as pd

In [ ]:
#import edges list

df = pd.read_csv('....csv')

In [ ]:
#create a dictionary showing users that engaged with each domain receiving more than 10 links

dict = {}

df2 = df.groupby('target').filter(lambda x: x['target'].count()>=10)

for host in pd.unique(df2['target']):
    dict[host] = set()
    for user in pd.unique(df2['source']):
        if len(df2[(df2['source']==user) & (df2['target']==host)]) != 0:
            dict[host].add(user)

In [ ]:
#create an edges list

edges_df = pd.DataFrame({'source': [],
                   'target': [],
                    'shared_audience': [],
                    'source_audience': [],
                   'source_audience_share': [],
                    'target_audience': [],
                   'target_audience_share': []})

for node in dict:
    for other_node in dict:
        if node != other_node and len(dict[node].intersection(dict[other_node])) != 0:
            index = len(edges_df.index)
            shared_audience = len(dict[node].intersection(dict[other_node]))
            edges_df.loc[index, 'source'] = node
            edges_df.loc[index, 'target'] = other_node
            edges_df.loc[index, 'shared_audience'] = shared_audience
            edges_df.loc[index, 'source_audience'] = len(df2[df2['target']==node])
            edges_df.loc[index, 'source_audience_share'] = shared_audience/len(df2[df2['target']==node])*100
            edges_df.loc[index, 'target_audience'] = len(df2[df2['target']==other_node])
            edges_df.loc[index, 'target_audience_share'] = shared_audience/len(df2[df2['target']==other_node])*100
            
decimals = pd.Series([2, 2], index=['source_audience_share', 'target_audience_share'])
edges_df = edges_df.round(decimals)
edges_df = edges_df.astype({'shared_audience': 'int', 'source_audience': 'int', 'target_audience': 'int'})
edges_df.to_csv('audience_share_edges_list.csv', index=False)

In [ ]:
edges_df.head(10).

In [ ]:
edges_df[['source', 'target', 'source_audience', 'shared_audience', 'source_audience_share']].sort_values(by='source_audience_share', ascending=False).head(10)

In [ ]:
#create a nodes list

nodes_list = list(set(list(edges_df['source']) + list(edges_df['source'])))

in_degrees= []

for i in nodes_list:
    in_degrees.append(len(df2[df2['target']==i]))
    
nodes_df = pd.DataFrame({'id': nodes_list,
                        'label': nodes_list,
                        'in_degrees': in_degrees})

nodes_df.to_csv('audience_share_nodes_list.csv', index=False)

In [ ]:
nodes_df.sort_values(by='in_degrees', ascending=False).head(10)